# 14-AzureAI-Agent Notebook

This notebook demonstrates how to interact with an Azure AI Agent. It shows how to create and manage agent runs, retrieve steps of each run, and manipulate messages generated by the agent for tasks such as Bing Search.

## Prerequisites
1. Valid Azure subscription  
2. Deploy an Azure AI Agent, you can create it with this [one click deployment](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2FAzure%2Fazure-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json)
- Note:Grounding with Bing Search only works with the following Azure OpenAI models: gpt-3.5-turbo-0125, gpt-4-0125-preview, gpt-4-turbo-2024-04-09, gpt-4o-0513, please choose a model from the list above
- It is recommended to choose model gpt-4o-0513
3. Create a Bing resource, and connect it to your agent, make sure to create the Bing resource under the same agent resource group, follow these [steps](https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/bing-grounding?tabs=python&pivots=overview#setup)
4. Insert the project connection string in .env file, You can find your connection string in the overview for your project in the Azure AI Foundry portal, under Project details > Project connection string.
5. Run az login


In [ ]:
%pip install azure-ai-projects azure-identity


In [19]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import BingGroundingTool


# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# It should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Login to Azure subscription via Azure CLI and set the environment variables

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)
   

In [ ]:
bing_connection = project_client.connections.get(
    connection_name="bin"
)
conn_id = bing_connection.id

print(conn_id)

# Initialize agent bing tool and add the connection id
bing = BingGroundingTool(connection_id=conn_id)

In [21]:

# Create agent with the bing tool and process assistant run
#with project_client:
agent = project_client.agents.create_agent(
    model="gpt-4o",
    name="bing-assistant",
    instructions="You are a helpful assistant",
    tools=bing.definitions,
    headers={"x-ms-enable-preview": "true"}
)
print(f"Created agent, ID: {agent.id}")
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="Can you summerizre this microsoft report https://www.microsoft.com/en-us/investor/earnings/fy-2023-q2/press-release-webcast" ,#"who is the president of the united states",
)
print(f"Created message, ID: {message.id}")
# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
print(f"Run finished with status: {run.status}")

# Retrieve run step details to get Bing Search query link
# To render the webpage, we recommend you replace the endpoint of Bing search query URLs with `www.bing.com` and your Bing search query URL would look like "https://www.bing.com/search?q={search query}"
run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)
run_steps_data = run_steps['data']
print(f"Last run step detail: {run_steps_data}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")


# Fetch and log all messages
messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages}")
print(messages.data)


Created agent, ID: asst_Wde6SoUg0Tsml9sCSO5KkvP2
Created thread, ID: thread_y5Qiyxg90v4wPTNNZCld8Z98
Created message, ID: msg_D2vN9OxAI9iBLJi18qzwPgd5
Run finished with status: RunStatus.COMPLETED
Last run step detail: [{'id': 'step_OVlbUd1agpytV7ANS3UDnnN2', 'object': 'thread.run.step', 'created_at': 1737638068, 'run_id': 'run_G25AweqXrKs1t1nZN4qf1Nvb', 'assistant_id': 'asst_Wde6SoUg0Tsml9sCSO5KkvP2', 'thread_id': 'thread_y5Qiyxg90v4wPTNNZCld8Z98', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1737638070, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_ybijc9b9S4VYLMqPBEzxS1NQ'}}, 'usage': {'prompt_tokens': 2387, 'completion_tokens': 212, 'total_tokens': 2599}}, {'id': 'step_3mHEPDzQLu2DQ1kAcrmK4oJA', 'object': 'thread.run.step', 'created_at': 1737638066, 'run_id': 'run_G25AweqXrKs1t1nZN4qf1Nvb', 'assistant_id': 'asst_Wde6SoUg0Tsml9sCSO5KkvP2', 'th

In [22]:
print(messages.data[0].content[0].text.value)


Microsoft's FY 2023 Q2 earnings report for the quarter ending December 31, 2022, highlights the following results compared to the prior year:

- **Revenue**: Increased 2% to $52.7 billion.
- **Operating Income**: Decreased 8% to $20.4 billion (GAAP).
- **Net Income**: Decreased 12% to $16.4 billion (GAAP).
- **Earnings per Share**: Decreased 11% to $2.20 (GAAP).

Key segments:
- **Productivity and Business Processes**: Revenue up 7% to $17.0 billion.
- **Intelligent Cloud**: Revenue up 18% to $21.5 billion.
- **More Personal Computing**: Revenue down 19% to $14.2 billion.

Microsoft Cloud revenue grew 22% to $27.1 billion. The company also returned $9.7 billion to shareholders via buybacks and dividends【3†source】.


In [24]:
# Delete the assistant when done
project_client.agents.delete_agent("asst_Wde6SoUg0Tsml9sCSO5KkvP2")
print("Deleted agent")

Deleted agent


In [25]:
print(project_client.agents.list_agents())

{'object': 'list', 'data': [{'id': 'asst_2pENUGFChHmUR8v6lCZggxB6', 'object': 'assistant', 'created_at': 1737638031, 'name': 'my-assistant', 'description': None, 'model': 'gpt-4o', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'bing_grounding', 'bing_grounding': {'connections': [{'connection_id': '/subscriptions/a1e02f1c-b501-4b1e-a9eb-af599ac3c567/resourceGroups/agent2/providers/Microsoft.MachineLearningServices/workspaces/project-demo-v2ex/connections/bin'}]}}], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}, {'id': 'asst_FNAo6PXvdK9q8mgMEQqOtwTY', 'object': 'assistant', 'created_at': 1737637058, 'name': 'my-assistant', 'description': None, 'model': 'gpt-4o', 'instructions': 'You are a helpful assistant', 'tools': [{'type': 'bing_grounding', 'bing_grounding': {'connections': [{'connection_id': '/subscriptions/a1e02f1c-b501-4b1e-a9eb-af599ac3c567/resourceGroups/agent2/providers/Microsoft.MachineLearningServices/wo

In [26]:
project_client.close()